In [1]:
import pandas as pd


# Read the cleaned file into a DataFrame
df_cleaned = pd.read_csv('../data/bgg_rating_threshold.csv', sep=';', engine='c')

# Convert the 'bgg_user_rating' column to numeric
df_cleaned['bgg_user_rating'] = pd.to_numeric(df_cleaned['bgg_user_rating'], errors='coerce')

print("done")

done


In [4]:
import numpy as np
# Create the matrix with games as rows and users as columns.
df_cleaned['bgg_user_rating'] = df_cleaned['bgg_user_rating'].astype(np.uint8)
#user_game_matrix = df_cleaned.head(1000000).pivot(index='bgg_id', columns='bgg_user_name', values='bgg_user_rating')
user_game_matrix = df_cleaned.set_index(['bgg_id', 'bgg_user_name'])['bgg_user_rating'].unstack('bgg_user_name',0)

# Check the shape and preview the matrix.
print(user_game_matrix.shape)

/tmp/ipykernel_59082/2928317859.py:5: PerformanceWarning: The following operation may generate 2499138456 cells in the resulting pandas object.
  user_game_matrix = df_cleaned.set_index(['bgg_id', 'bgg_user_name'])['bgg_user_rating'].unstack('bgg_user_name',0)


(6126, 407956)


In [5]:
import numpy as np
import scipy
sparse = scipy.sparse.csr_matrix(user_game_matrix.values)

u4, s4, vT4= scipy.sparse.linalg.svds(sparse,return_singular_vectors='u', k=3000, solver='propack')
#u4, s4, vT4= scipy.sparse.linalg.svds(sparse, k=200)
np.savetxt("../data/features.csv", u4, delimiter=";")

: 

In [ ]:
# Reconstruct the approximation from the SVD factors.
reconstructed = u4 @ np.diag(s4) @ vT4

# # Compute the Frobenius norm of the difference.
# error = np.linalg.norm(sparse.toarray() - reconstructed, ord='fro')

# # And the Frobenius norm of the original matrix.
# norm_orig = np.linalg.norm(sparse.toarray(), ord='fro')

# relative_error = error / norm_orig
# print("Relative Frobenius error:", relative_error)

In [6]:
np.allclose(sparse.toarray(), reconstructed)

: 